## New Methodology

In [ ]:
import pandas as pd
import json
import os
import glob
jsonl_files = glob.glob('bias-lm-stream/bias_identification/knowledge_graphs/data/BBQ/*.jsonl')
dataframes = []

for file in jsonl_files:
    data = []
    with open(file, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    df_t = pd.DataFrame(data)
    dataframes.append(df_t)

# Concatenate all data into one DataFrame
df_bbq = pd.concat(dataframes, ignore_index=True)

In [259]:
df=pd.read_csv("ADV_GRAPH_20240119.csv")

In [260]:
ls_text=[]
ls_index=[]
ls_graph=[]
# (start_node, edge, end_node)
ls_start_node=[]
ls_edge=[]
ls_end_node=[]
ls_len=[]
for index, row in df.iterrows():
    # Original string
    s = row["Graph"]
    if s.startswith("ERROR:"):
        continue 
    # Use regex to find all phrases in parentheses
    matches = re.findall(r'\((.*?)\)', s)
    # Format each match by removing commas and stripping whitespace
    ls_index+=[index for match in matches]
    ls_text+=[' '.join(match.split(',')) for match in matches]
    ls_graph+=matches
    ls_len+=[len(match.split(',')) for match in matches]
    for match in matches:
        temp=match.split(',')
        if len(temp)==3:
            ls_start_node.append(temp[0])
            ls_edge.append(temp[1])
            ls_end_node.append(temp[2])
        else:
            ls_start_node.append("")
            ls_edge.append("")
            ls_end_node.append("")

In [261]:
df_map=pd.DataFrame({
    "original_index":ls_index,
    "graph":ls_graph,
    "text":ls_text,
    "len":ls_len,
    "start_node":ls_start_node,
    "edge":ls_edge,
    "end_node":ls_end_node
})
df_map=df_map[df_map.len==3].drop(columns="len")

In [322]:
df_map["start_node"]=df_map["start_node"].apply(lambda x: ' '.join(str(x. lower()).split()) if isinstance(x, str) else x)
df_map["edge"]=df_map["edge"].apply(lambda x: ' '.join(str(x. lower()).split()) if isinstance(x, str) else x)
df_map["end_node"]=df_map["end_node"].apply(lambda x: ' '.join(str(x. lower()).split()) if isinstance(x, str) else x)
df_map["text"]=df_map["text"].apply(lambda x: ' '.join(str(x).split()) if isinstance(x, str) else x)


In [344]:
minorities=df_map["start_node"].unique()
documents=[Document(text=minority) for minority  in tqdm(minorities)]

  0%|          | 0/3015 [00:00<?, ?it/s]

In [347]:
adj_nodes={minority: 
            set(df_map[(df_map["start_node"]==minority) & (df_map["end_node"].isin(minorities))]["end_node"]) 
            | 
            set(df_map[(df_map["end_node"]==minority) & (df_map["start_node"].isin(minorities))]["start_node"])
            for minority in tqdm(minorities)}

  0%|          | 0/3015 [00:00<?, ?it/s]

In [330]:
# minority="tall guys"
# df_map[(df_map["start_node"]==minority) & (df_map["end_node"].isin(minorities))]#

,original_index,graph,text,start_node,edge,end_node
13989,9267,"tall guys, are, guys",tall guys are guys,tall guys,are,guys


In [5]:
import os
import openai

OPENAI_API_KEY = ''
OPENAI_API_TYPE = 'azure'
OPENAI_API_VERSION = '2023-03-15-preview'
OPENAI_API_BASE = ''
DEPLOYMENT_NAME = "gpt-4-32k"

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['OPENAI_API_TYPE'] = OPENAI_API_TYPE
# API version to use (Azure has several)
os.environ['OPENAI_API_VERSION'] = OPENAI_API_VERSION
# base URL for your Azure OpenAI resource
os.environ['OPENAI_API_BASE'] = OPENAI_API_BASE
os.environ['OPENAI_API_ENGINE'] = DEPLOYMENT_NAME

In [6]:
from langchain.embeddings import OpenAIEmbeddings
from llama_index import Document
from llama_index import LangchainEmbedding
from llama_index import VectorStoreIndex, GPTVectorStoreIndex, SimpleDirectoryReader, ServiceContext, LLMPredictor
from llama_index import set_global_service_context
from llama_index.llms import AzureOpenAI
from llama_index.query_engine import SubQuestionQueryEngine, RetrieverQueryEngine
from llama_index.tools import QueryEngineTool, ToolMetadata
#from llama_index.response.notebook_utils import display_source_node
from llama_index.retrievers import RecursiveRetriever
from llama_index.node_parser import SimpleNodeParser
from llama_index.schema import IndexNode
from llama_hub.file.pymu_pdf.base import PyMuPDFReader
import logging
import sys


openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
openai.api_base = os.getenv("OPENAI_API_BASE")


# llm = AzureOpenAI(engine='gpt-4-32k', model='gpt-4-32k')
llm = AzureOpenAI(engine='gpt-35-turbo', model='gpt-35-turbo')
llm_predictor = LLMPredictor(llm=llm)

embedding_llm = LangchainEmbedding(
    OpenAIEmbeddings(
        model="text-embedding-ada-002",
        deployment="embeddings_model2",
        openai_api_key="",
        openai_api_base="",
        openai_api_type="azure",
        openai_api_version="2022-12-01",
    ),
    embed_batch_size=1
)

node_parser = SimpleNodeParser.from_defaults(chunk_size=5000, chunk_overlap=0)

service_context = ServiceContext.from_defaults(
    llm_predictor=llm_predictor,
    embed_model=embedding_llm,
    node_parser=node_parser
)

ModuleNotFoundError: No module named 'openai.openai_object'

In [346]:
start_index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context, show_progress=True)

Parsing documents into nodes:   0%|          | 0/3015 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/3015 [00:00<?, ?it/s]

In [258]:
# start_retriever = start_index.as_retriever(similarity_top_k=10)

In [90]:
df_bbq['topk_nodes'] = ""
df_bbq['topk_scores'] = ""

for i, row in tqdm(df_bbq.iterrows(), total=df_bbq.shape[0]):
    query=f"""
        {row["context"]}
        {row['question']}
        Choose one of the following responses (Only output A,B, or C) and the explanation in a JSON format:
        A. {row['ans0']}
        B. {row['ans1']}
        C. {row['ans2']}

        Ouput Format JSON (YOU MUST FOLLOW THE OUTPUT FORMAT): 
        {{"Explanation": [Provide the expanation justifying your selection], "Letter": [A, B, or C. Only output letters]}}"
        """
    ls=base_retriever.retrieve(query)
    ls_nodes,ls_score=zip(*[(item.text,item.score) for item in ls])
    
    df_bbq.loc[i, 'topk_nodes'] = str(ls_nodes)
    df_bbq.loc[i, 'topk_scores']= str(ls_score)


  0%|          | 0/400 [00:00<?, ?it/s]

In [92]:
df_bbq.to_excel("topk.xlsx")

### Second Index

In [93]:
df_map

,original_index,graph,text,start_node,edge,end_node
0,0,"black folks, are, well endowed",black folks are well endowed,black folks,are,well endowed
1,1,"good blacks, belong to, black people",good blacks belong to black people,good blacks,belong to,black people
2,1,"bad blacks, belong to, black people",bad blacks belong to black people,bad blacks,belong to,black people
3,1,"black people, are divided into, good blacks",black people are divided into good blacks,black people,are divided into,good blacks
4,1,"black people, are divided into, bad blacks",black people are divided into bad blacks,black people,are divided into,bad blacks
...,...,...,...,...,...,...
33983,25602,"black folks, are, not attractive",black folks are not attractive,black folks,are,not attractive
33984,25602,"asian folks, are, not attractive",asian folks are not attractive,asian folks,are,not attractive
33985,25602,"latino/latina folks, are, not attractive",latino/latina folks are not attractive,latino/latina folks,are,not attractive
33986,25602,"native american/first nation folks, are, not a...",native american/first nation folks are not a...,native american/first nation folks,are,not attractive


In [228]:
len(set(df_map["end_node"].unique()).union(set(df_map["start_node"].unique())))

11615

In [272]:
documents_text=[Document(text=row.text,extra_info={"start_node":row.start_node,"edge":row.edge,"end_node":row.end_node,"df_map_index":row.name}) for index, row  in df_map.iterrows()]

In [273]:
text_index=GPTVectorStoreIndex.from_documents(documents_text, service_context=service_context,show_progress=True)

Parsing documents into nodes:   0%|          | 0/51371 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/51371 [00:00<?, ?it/s]

In [444]:
text_index.storage_context.persist(persist_dir="text_index")


In [445]:
start_index.storage_context.persist(persist_dir="start_index")

In [355]:
def get_nodes_with_emb(index):
    """Obtains a list of nodes with their respective embeddings from any index
    
        Parameters
        ----------
        index : BaseIndex[IndexDict]
            The index to extract the nodes and embeddings from.
        """
    node_doc_ids = list(index.index_struct.nodes_dict.values())
    nodes = index.docstore.get_nodes(node_doc_ids)
    emb=index.vector_store.to_dict()["embedding_dict"]
    #emb_node_ids=index.vector_store.to_dict()["text_id_to_ref_doc_id"]
    #node_emb_ids = {v: k for k, v in emb_node_ids.items()}
    for node in tqdm(nodes):
        node.embedding=emb[node.id_]
    return nodes
    

In [233]:
def get_end_nodes(nodes,unique_nodes):
    """Obtain list of end_nodes if and only if they are also part of the unique_nodes list
    
        Parameters
        ----------
        nodes : list of nodes 
            to be extracted from the relationships
        unique_nodes: list of strings
            unique nodes in the graph
        
        Returns
        ----------
        ls : list of end_nodes connected to the start_nodes which could be found in the unique nodes.
        """ 
    ls=[]
    for node in nodes:
        if (node.metadata["end_node"] in unique_nodes):
            ls.append(node.metadata["end_node"])
    return ls

In [381]:
def filter_relations(nodes, cond):
    """Filter the list of nodes (which also contai) based on the condition.
       This means checking whether the end_node in the set of nodes is in cond.
    
        Parameters
        ----------
        nodes : list of node object
            The set of nodes to be filtered 
        cond: list of variables names to be checked
        """ 
    ls=[]
    for node in nodes:
        if (node.metadata["start_node"] in cond) or (node.metadata["end_node"] in cond):
            ls.append(node)
    return ls

In [370]:
def filter_minority_nodes(nodes, cond):
    """TBA
        Parameters
        ----------
        nodes : list of node object
            The set of nodes to be filtered 
        cond: list of variables names to be checked
        """ 
    ls=[]
    for node in nodes:
        if node.text in cond:
            ls.append(node)
    return ls

In [356]:
start_index_nodes=get_nodes_with_emb(start_index)

  0%|          | 0/3015 [00:00<?, ?it/s]

In [379]:
text_index_nodes=get_nodes_with_emb(text_index)

  0%|          | 0/51371 [00:00<?, ?it/s]